In [3]:
import pandas as pd
import numpy as np
import psycopg2

from sklearn.preprocessing import RobustScaler
import tsmoothie 

In [4]:
def data_extractor(asset,cols=['DCP','DNCP','OPCP','HPCP','LPCP','CPCP','ACPCP','VTCP','MPN5P']):
    # The credentials to conect to the database
    hostname = 'database-1.ctzm0hf7fhri.eu-central-1.rds.amazonaws.com'
    database = 'dyDATA_new'
    username = 'postgres'
    pwd = 'Proc2023awsrdspostgresql'
    port_id = 5432
    conn = None
    asset_script="SELECT * FROM "+'\"'+"ASSET_"+asset+'\"'+".features_targets_input_view WHERE features_targets_input_view."+'\"'+"cleaned_raw_features_environment_PK"+'\"'+ "= 4"
    # Here we select the active financial asset from the financial asset list table
    try:
        with psycopg2.connect(
            host = hostname,
            dbname = database,
            user = username,
            password = pwd,
            port = port_id
        ) as conn:
            dataframe = pd.read_sql(asset_script,conn)
    except Exception as error:
        conn.close()
        return error
    finally:
        if conn is not None:
            conn.close()
    dataframe = dataframe.filter(regex='|'.join(cols),axis=1)
    
    for i,j in zip(cols,dataframe.columns):
        dataframe.rename(columns={j:i},inplace=True)

    return dataframe

In [5]:
df = data_extractor(asset='MSFT')
df.set_index('DCP', inplace=True) 
features = ['OPCP', 'HPCP', 'LPCP', 'CPCP', 'ACPCP', 'VTCP']# 'F1', 'F2', 'F3'] 
target = 'MPN5P'
df = df[features + [target]]
df.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_7964\1301242928.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(asset_script,conn)


AttributeError: 'DatabaseError' object has no attribute 'set_index'

In [322]:
train_start = '2000-01-01'
train_end = '2021-12-31'
test_start = '2022-01-01'
test_end = '2022-12-30'

train = df.loc[(df.index >= train_start) & (df.index <= train_end)]
test = df.loc[(df.index >= test_start) & (df.index <= test_end)]

In [323]:
X_train = train.drop("MPN5P", axis=1)
y_train = train["MPN5P"]

X_test = test.drop("MPN5P", axis=1)
y_test = test["MPN5P"]

In [324]:
y_test

DCP
2022-01-03    314.7200
2022-01-04    314.1500
2022-01-05    314.1500
2022-01-06    314.1500
2022-01-07    313.3800
                ...   
2022-12-23    238.5575
2022-12-27    237.4000
2022-12-28    236.6600
2022-12-29    230.0525
2022-12-30    227.2000
Name: MPN5P, Length: 251, dtype: float64

In [325]:
def log_transform(dataframe):
    transformed_df = np.log(dataframe)
    transformed_df = transformed_df.fillna(-1)
    return transformed_df

kalman_smoother = tsmoothie.KalmanSmoother(component='level_trend', component_noise={'level':0.1, 'trend':0.1})
def kalman_filter(dataframe):
    dataframe = pd.DataFrame(dataframe)
    smoothed_data = pd.DataFrame()
    for column in dataframe.columns:
        smoothed_data[column] = pd.Series(kalman_smoother.smooth(dataframe[column]).smooth_data.flatten(), index=dataframe.index)
    return smoothed_data 

### Log Transform

In [326]:
X_train = np.log(X_train)
y_train = np.log(y_train)

X_test = np.log(X_test)
y_test = np.log(y_test)

In [327]:
X_train

,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP
DCP,,,,,,
2000-01-03,4.072227,4.082820,4.025352,4.065280,3.591329,17.790103
2000-01-04,4.039206,4.070095,4.027581,4.030917,3.556965,17.806696
2000-01-05,4.017509,4.063671,4.001635,4.041405,3.567453,17.975324
2000-01-06,4.027024,4.041954,3.992450,4.007333,3.533382,17.822418
2000-01-07,3.994754,4.027581,3.982598,4.020317,3.546365,17.942864
...,...,...,...,...,...,...
2021-12-27,5.815503,5.836213,5.815413,5.836126,5.822114,16.808589
2021-12-28,5.838168,5.840089,5.829886,5.832615,5.818604,16.566716
2021-12-29,5.832762,5.841513,5.828004,5.834665,5.820653,16.526357


### Kalman Filter

In [328]:
X_train = kalman_filter(X_train)
y_train = kalman_filter(y_train)

X_test = kalman_filter(X_test)
y_test = kalman_filter(y_test)

### Robust Scaler

In [329]:
scaler_features = RobustScaler().fit(X_train)
scaler_target = RobustScaler().fit(y_train)

In [330]:
X_train = pd.DataFrame(scaler_features.transform(X_train), columns = features)
y_train = pd.DataFrame(scaler_target.transform(y_train), columns = [target])

In [331]:
X_test = pd.DataFrame(scaler_features.transform(X_test), columns = features)
y_test = pd.DataFrame(scaler_target.transform(y_test), columns = [target])

### Post Treatment

In [332]:
y_test = scaler_target.inverse_transform(y_test)
y_test = np.exp(y_test)

y_test = pd.DataFrame(y_test, columns=[target])
y_test

,MPN5P
0,277.078549
1,292.245575
2,303.103834
3,309.469859
4,312.023011
...,...
246,237.161548
247,235.920113
248,234.024926
249,231.419488
